In [3]:
%matplotlib notebook 
import scipy.stats as stats
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm
from scipy import integrate
import glob,sys,os

from mpl_toolkits.basemap import Basemap, shiftgrid

import MV2 as MV
import cdms2 as cdms
import genutil
import cdutil
import cdtime

sys.path.append("/Users/kmarvel/Google Drive/python-utils")
import Plotting

import requests
import pandas as pd

### Set classic Netcdf (ver 3)
cdms.setNetcdfShuffleFlag(0)
cdms.setNetcdfDeflateFlag(0)
cdms.setNetcdfDeflateLevelFlag(0)

In [2]:
curr_mod="CESM2"
curr_var="pr"
experiment_id="piControl"
member_id="r1i1p1f1"

In [4]:
def pull_data(curr_mod,curr_var,experiment_id,member_id):
    # Baseline directory
    base_dir   = 'http://mary.ldeo.columbia.edu:81/CMIP6/.'
    # Write directory
    base_write_dir = '/Users/kmarvel/Documents/DATA/dester/piControlRaw/'

    df_proclist = pd.DataFrame(columns=['model','sim','ensemble','variable'])
    ingrid_cmip6 = pd.read_csv('mary_cmip6_101419.csv')


    write_dir = base_write_dir+curr_var+"/"+curr_mod+"/"
    #If the directory doesn't exist already, make it
    os.makedirs(os.path.join(base_write_dir, curr_var, curr_mod),exist_ok=True)
    #rips=np.unique(np.array(df1.member_id))

    df1 = ingrid_cmip6[(ingrid_cmip6.source_id==curr_mod)\
                                   & (ingrid_cmip6.variable_id==curr_var) & \
                                   (ingrid_cmip6.experiment_id == experiment_id)&\
                                  (ingrid_cmip6.member_id == member_id)]

    #Construct openDAP link
    nfiles,nidentifiers=df1.shape
    times=np.sort(np.array(df1.time_range))
    i_ens=np.where(df1.time_range==times[0])[0]
    time_range=times[0]

    for time_range in times:
        # Construct Remote OpenDAP Link
        i_ens=np.where(df1.time_range==time_range)[0][0]
        nc_link = base_dir+df1.activity_id.iloc[i_ens]+'/.'+df1.institution_id.iloc[i_ens]+'/.'+curr_mod+'/.'+experiment_id+'/.'+df1.member_id.iloc[i_ens]+'/.'+df1.table_id.iloc[i_ens]+ \
                            '/.'+curr_var+'/.'+df1.grid_label.iloc[i_ens]+'/.'+df1.version.iloc[i_ens]+'/.'+df1.file_basename.iloc[i_ens]+'/.'+curr_var+'/dods'
        request = requests.get(nc_link)
        if request.status_code == 200:
            #Get the data
            f=cdms.open(nc_link)
            data=f(curr_var)
            tax=data.getTime()

            #reshape it to years and months
            ntime=data.shape[0]
            nyears=int(ntime/12)
            rdata=data.reshape((nyears,12)+data.shape[1:])
            for i in range(nyears):
                yeardata=rdata[i]
                #Make the time axis
                tax_trunc=cdms.createAxis(tax[12*i:12*(i+1)])
                tax_trunc.designateTime()
                for key in tax.attributes.keys():
                    setattr(tax_trunc,key,tax.attributes[key])
                yeardata.setAxis(0,tax_trunc)
                #get the start year for labeling purposes
                year=str(tax_trunc.asComponentTime()[0].year)
                writename = curr_var+"."+experiment_id+"."+curr_mod+"."+member_id+"."+year.zfill(4)+".nc"
                fw=cdms.open(write_dir+writename,"w")
                fw.write(yeardata)
                fw.close()
            f.close()


In [ ]:
curr_mod="BCC-CSM2-MR"
curr_var="pr"
experiment_id="piControl"
member_id="r1i1p1f1"
pull_data(curr_mod,curr_var,experiment_id,member_id)

In [6]:
ingrid_cmip6 = pd.read_csv('mary_cmip6_101419.csv')

In [7]:
variables=["pr","evspsbl","mrro","mrros","mrso","mrsos","pr","prsn","tas"]
models=np.unique(ingrid_cmip6.source_id)
for variable in variables:
    for model in models:
        try:
            pull_data(model,variable,"piControl","r1i1p1f1")
        except:
            print(variable+" and "+ model +" are bad")

pr and AWI-CM-1-1-MR are bad


/Users/kmarvel/anaconda3/envs/cdat_plus/lib/python3.6/site-packages/numpy/ma/core.py:3169: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  dout = self.data[indx]


pr and CNRM-CM6-1 are bad
pr and CNRM-ESM2-1 are bad
pr and EC-Earth3 are bad
pr and EC-Earth3-LR are bad
pr and EC-Earth3-Veg are bad
pr and FGOALS-f3-L are bad
pr and FGOALS-g3 are bad
pr and GFDL-AM4 are bad
pr and GISS-E2-1-G-CC are bad
pr and MCM-UA-1-0 are bad
pr and MIROC-ES2L are bad
pr and MPI-ESM1-2-HR are bad
pr and NESM3 are bad
pr and UKESM1-0-LL are bad
evspsbl and AWI-CM-1-1-MR are bad
evspsbl and CNRM-CM6-1 are bad
evspsbl and CNRM-ESM2-1 are bad
evspsbl and EC-Earth3 are bad
evspsbl and EC-Earth3-LR are bad
evspsbl and EC-Earth3-Veg are bad
evspsbl and FGOALS-f3-L are bad
evspsbl and FGOALS-g3 are bad
evspsbl and GFDL-AM4 are bad
evspsbl and GFDL-ESM4 are bad
evspsbl and GISS-E2-1-G-CC are bad
evspsbl and MCM-UA-1-0 are bad
evspsbl and MIROC-ES2L are bad
evspsbl and MPI-ESM1-2-HR are bad
evspsbl and NESM3 are bad
evspsbl and UKESM1-0-LL are bad
mrro and AWI-CM-1-1-MR are bad


/Users/kmarvel/anaconda3/envs/cdat_plus/lib/python3.6/site-packages/numpy/ma/core.py:3201: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  mout = _mask[indx]


mrro and CNRM-CM6-1 are bad
mrro and CNRM-ESM2-1 are bad
mrro and EC-Earth3 are bad
mrro and EC-Earth3-LR are bad
mrro and EC-Earth3-Veg are bad
mrro and FGOALS-f3-L are bad
mrro and FGOALS-g3 are bad
mrro and GFDL-AM4 are bad
mrro and GISS-E2-1-G-CC are bad
mrro and MCM-UA-1-0 are bad
mrro and MIROC-ES2L are bad
mrro and MPI-ESM1-2-HR are bad
mrro and NESM3 are bad
mrro and UKESM1-0-LL are bad
mrros and AWI-CM-1-1-MR are bad
mrros and CNRM-CM6-1 are bad
mrros and CNRM-ESM2-1 are bad
mrros and EC-Earth3 are bad
mrros and EC-Earth3-LR are bad
mrros and EC-Earth3-Veg are bad
mrros and FGOALS-f3-L are bad
mrros and FGOALS-g3 are bad
mrros and GFDL-AM4 are bad
mrros and GISS-E2-1-G-CC are bad
mrros and MCM-UA-1-0 are bad
mrros and MIROC-ES2L are bad
mrros and MPI-ESM1-2-HR are bad
mrros and NESM3 are bad
mrros and UKESM1-0-LL are bad
mrso and AWI-CM-1-1-MR are bad
mrso and CNRM-CM6-1 are bad
mrso and CNRM-ESM2-1 are bad
mrso and EC-Earth3 are bad
mrso and EC-Earth3-LR are bad
mrso and EC-E